In [ ]:
import math

In [ ]:
import pandas as pd

import numpy as np

import nltk

import re
from collections import Counter

import string

import pprint

In [ ]:
from statistics import mean

In [ ]:
from itertools import islice

In [ ]:
import scipy

Nb Formatting 

In [ ]:
pd.options.display.max_seq_items = 2000

pd.set_option('display.max_columns', 100)

# Data setup

Open Dataset 

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
#ignore header for easier (ordinal) indexing
datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

In [ ]:
len(datasheet)

In [ ]:
datasheet.head()

Row Indices of Memory Responses 

In [ ]:
#ignore first (header) column
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index


Record IDs of Memory Respondents [Analyze all scores from each participant]

In [ ]:
memory_id = list(datasheet.iloc[memory_ix][0])

Confirm Unique Response-Respondents 

In [ ]:
len(memory_id)

In [ ]:
len(set(memory_id))

In [ ]:
datasheet.iloc[memory_ix]

Tech respondents

In [ ]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [ ]:
tech_ix

In [ ]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [ ]:
len(tech_id)

In [ ]:
#each respondent responded to one prompt
len(tech_id) == len(set(tech_id))

In [ ]:
ix_id_series = pd.Series(tech_id,index=tech_ix)

In [ ]:
ix_id_series

Tech-exclusive respondents

In [ ]:
#row indices of tech-respondents, mem-negligent
tech_x_ix = [tix for tix in tech_ix if tix not in memory_ix]

In [ ]:
#record ids of tech-respondents who are mem-negligent
tech_x_ids = [tid for tid in tech_id if tid not in memory_id]

Mem-exclusive respondents

In [ ]:
#mem respondents who are tech-negligent
[m for m in memory_id if m not in tech_id]

Indices of all Respondent-rows, by record ID

In [ ]:
#all rows for tech_respondents
tech_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in tech_id]

In [ ]:
max(datasheet[datasheet[0]==14].index)

In [ ]:
datasheet.iloc[tech_all_ix]

In [ ]:
mem_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in memory_id]

In [ ]:
datasheet.iloc[mem_all_ix]

Column index Dict

Investigate score spread

In [ ]:
datasheet.iloc[tech_ix,51:54]

In [ ]:
#MMSE
datasheet.iloc[tech_all_ix][[0,1,4,74]]

In [ ]:
#MMSE scores
#6 unique...
np.unique([int(i) for i in datasheet.iloc[tech_all_ix][74].dropna()],return_counts=True)

In [ ]:
#Min MMSE
datasheet.iloc[tech_all_ix][74].min()

In [ ]:
#The maximum MMSE score is 30 points. A score of 20 to 24 suggests mild dementia, 13 to 20 suggests moderate dementia, and less than 12 indicates severe dementia.
#participants with MCI...one ---
datasheet.iloc[tech_all_ix][74][datasheet.iloc[tech_all_ix][74] < 25]

In [ ]:
#Unique words

In [ ]:
len(tech_ix)

## Lancet dataset

In [ ]:
lancet = pd.read_excel('Lancet_82318.xlsx',header=None)

In [ ]:
[(i,name) for (i,name) in enumerate(lancet.iloc[0])]

In [ ]:
lancet_id = set(lancet[0][1:])

In [ ]:
#lancet dataset accounts for all respondents 
[no_lance for no_lance in tech_id if no_lance not in lancet_id]

In [ ]:
lancet.index

In [ ]:
#indices in lancet dataset for which record_id responded to either/both prompts
lancet_all_ix = [ix for ix in lancet.index if lancet.loc[ix][0] in tech_id]

In [ ]:
lancet_id_ix = lancet.iloc[lancet_all_ix,[0,17]]

In [ ]:
#each participant has one diagnosis row, other rows are na --> retain one diagnosis per patient 
lancet_id_ix = lancet_id_ix.dropna()

In [ ]:
#dataset breakdown 
lancet_id_ix[17].value_counts()

In [ ]:
len(lancet_id_ix) == len(ix_id_series)

## Metadata

Gender

In [ ]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Female')[0])

In [ ]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Male')[0])

Age

In [ ]:
lancet.iloc[lancet_all_ix][5].describe()

In [ ]:
mean(lancet.iloc[lancet_all_ix][5].dropna())

In [ ]:
max(lancet.iloc[lancet_all_ix][5].dropna())

In [ ]:
min(lancet.iloc[lancet_all_ix][5].dropna())

In [ ]:
lancet.iloc[520:530]

Diagnosis

In [ ]:
lancet.iloc[lancet_all_ix][[16,17]].dropna()

In [ ]:
np.where(min(lancet.iloc[lancet_all_ix][5].dropna()))

# Reference

## Column names

In [ ]:
datasheet_col_ix_name = [(i,test) for( i,test) in enumerate(datasheet.iloc[0])]

## 466k Word list

In [ ]:
#https://raw.githubusercontent.com/dwyl/english-words/master/words.txt
with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]

In [ ]:
len(word_list)

## nltk pos data dictionary

In [ ]:
nltk.help.upenn_tagset()

# Feature Extraction

In [ ]:
#memory response
mem_corpus = datasheet.iloc[memory_ix,2]

In [ ]:
#tech response
tech_corpus = datasheet.iloc[tech_ix,3]

# Helper Functions

### try-catch comprehension hack
https://stackoverflow.com/questions/1528237/how-can-i-handle-exceptions-in-a-list-comprehension-in-python

In [ ]:
def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    
    except ZeroDivisionError as z:
        return None
    except statistics.StatisticsError:
        print('Stat error at ix')
        return handle(e)
    except Exception as e:
        print(handle(e))
        return None
        #return handle(e)

## Tokenization

In [ ]:
#Mis-spellings, word length
#rm puctuation
#keep contractions, posessives
def list_tokenize(response):
    return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()

## Information extraction nltk

In [ ]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

## Uniq, V

In [ ]:
def uniq_words (response,token_list=None, counts_vector_return=False,uniq_set_return=False):
    if token_list is None: token_list = list_tokenize(response)
    if counts_vector_return and uniq_set_return: return np.unique(token_list,return_counts=True)
    
    if counts_vector_return and not uniq_set_return: return np.unique(token_list,return_counts=True)[1]
    
    if uniq_set_return and not counts_vector_return: return np.unique(token_list)
    #typical case, investigating number of uniq responses
    
    return len(np.unique(token_list))
        

## V1 

In [ ]:
def single_appearances (response,counts=None):
    if counts is None: counts = uniq_words(response,counts_vector_return=True)
    return len(np.where(counts==1)[0])

## N-grams

In [ ]:
#http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
#input
def find_bigrams(response, return_list=False):
    input_list = list_tokenize(response)
    bigrams = zip(input_list, input_list[1:])
    if return_list: return [b for b in bigrams]
    else: return bigrams

## Word count

In [ ]:
def word_count(response):
    return len(list_tokenize(response))

## Sentence count

In [ ]:
def sentence_count(response):
    return len(ie_preprocess(response))

### POS tagging

In [ ]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

In [ ]:
nltk.help.upenn_tagset()

## Rates

In [ ]:
#this method is error-prone
def pos_rate(response,pos_response=None,tag_set=None,denom_set=None,N=None,nn=False,pn=False,vb=False,cloc=False):
    
    if pos_response is None: pos_response = ie_preprocess(response)
        
    
    if tag_set is None:
        #common nouns, excluding proper nouns
        if nn: tag_set = ['NN','NNS']
        elif vb: tag_set = ['VB','VBD','VBG','VBN','VBP','VBZ'] 
        elif pn: tag_set = ['PRP','PRP$']
            #engineer: include all specific nouns, unspecified nouns 'stuff' 'things' 'this' in denom -- list better than intuition?
            #consider removing self-reference pronouns or count separately?
        elif cloc: 
            tag_set = ['PRP','PRP$','WP', 'DT','WDT','WP$']
            #amend-?
#             thing_set = ['thing','things','stuff','anything','everything','something']
            denom_set = ['NN','NNP','NNPS','NNS']
#         elif: lexical_density = [] n adj vb advb 
        else: 
            print('Usage: set pos rate metric to True')
            return None
        
    pos_count = sum([1 for sentence in pos_response for word in sentence if word[1] in tag_set])
    
    if denom_set: 
#         print('Thing_set',[word[0] for sentence in pos_response for word in sentence if word[0] in thing_set])
#         print('Pronouns',[word[0] for sentence in pos_response for word in sentence if word[1] in tag_set])
#         pos_count += sum([1 for sentence in pos_response for word in sentence if word[0] in thing_set])
        denom_count = sum([1 for sentence in pos_response for word in sentence if word[1] in denom_set])
        if denom_count == 0: return None
    
    else:
        if N is None: N = word_count(response)
        denom_count = N
        
    return pos_count/denom_count
        

#         sum([1 for sentence in pos_test for word in sentence if word[1] in ['VB','VBD','VBG','VBN','VBP','VBZ'] ])

In [ ]:
datasheet.iloc[522,3]

## Function-word rate

In [ ]:
from nltk.corpus import stopwords
stopwords.ensure_loaded()
swords = stopwords.words()

def func_rate(response, words=swords):
    response = list_tokenize(response)
    N = len(response)
#     print([word for word in response if word in swords])
    func_count = sum([1 for word in response if word in swords])
    return func_count/N

In [ ]:
func_rate(datasheet.iloc[memory_ix[8],2])

## Honore's R 
![image.png](attachment:image.png)
https://books.google.com/books?id=CwC4CwAAQBAJ&pg=PA111&lpg=PA111&dq=honore+statistic+lexical+richness&source=bl&ots=LPnM2j9oX5&sig=HGObkoMYRy6lLsgc80Y7tUMT7vk&hl=en&sa=X&ved=2ahUKEwjv0ubE7vTdAhXwYd8KHVRfCiMQ6AEwCXoECAUQAQ#v=onepage&q=honore&f=false

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.2.1359&rep=rep1&type=pdf

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.484.4648&rep=rep1&type=pdf

viii) Honoré’s R statistic.
If V1 denotes the number of hapax legomena recorded, then Honoré (1979) defined his
statistic as:

*R = 100log(N)/(1 - V1/V)*

As a measure of vocabulary richness which has the virtue of being insensitive to text
length, it has been used in stylometric studies by Holmes (1992) and Holmes and
Stylometric analysis of conversational speech
Forsyth (1995). Values of R typically range from 1000 to 2000 with higher values
implying richer vocabularies in the sense that a greater number of words appear
infrequently.


https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf


Assuming log == base e...fits with 1000-2000 assumption




In [ ]:
def honore_r(response,v=None,v1=None, N=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_words(response)
    if v1 is None: 
        counts = uniq_words(response,counts_vector_return=True)
        v1 = len(np.where(counts==1)[0])
    return (100*math.log(N))/(1-(v1/v))

In [ ]:
# mem_honore = pd.Series([catch(lambda:honore_r(response)) for response in datasheet.iloc[memory_ix,2]])

In [ ]:
# tech_honore = pd.Series([catch(lambda:honore_r(response)) for response in datasheet.iloc[tech_ix,3]])

In [ ]:
mem_honore

## Brunet's W
W = N**(V −0.165)

(vii) Brunet’s W index.
This index, devised by Brunet (1978) and used successfully by Holmes and Forsyth
(1995), is a measure of vocabulary richness which is insensitive to text length. It is
defined as:

W= N^(V-0.165)
where N is the text length, V the number of different words and (-0.165) is a scaling
constant proposed by Brunet. The measure generally varies between 10 and 20 with a
low value indicating a lexically richer speech.
https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf

In [ ]:
def brunet_w(response,N=None, v=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_words(response)
    return N**(v-0.165)

## Moving window

In [ ]:
#10-window size generator
#https://docs.python.org/release/2.3.5/lib/itertools-example.html
def window(seq, n=10):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [ ]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]),8)]

|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|

### Indices to pull other target-options in clinic dataset

In [ ]:
#indices to grab latest cog scores
ix_score_latest = [max(datasheet[datasheet[0]==ix].index) for ix in tech_id]

In [ ]:
ix_score_latest_completed = []
for ix in tech_id:
    slice_ix = list(datasheet[datasheet[0]==ix].index)
    while datasheet.iloc[max(slice_ix)][96] != 'Complete':
        slice_ix.remove(max(slice_ix))
    ix_score_latest_completed += [max(slice_ix)]

# Held-out test set

np.random.choice(tech_ix,(1,int(.20*len(tech_ix))),replace=False)

array([[ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375]], dtype=int64)

In [ ]:
held_out_test_ix = np.array([ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375])

In [ ]:
held_out_test_id = [i for i in datasheet.iloc[held_out_test_ix][0]]

In [ ]:
#held out test-set coincidently has both tech + mem responses
#dev set has one tech_x responose
tech_x_ix

In [ ]:
y_held_out_test_ix = [yt for yt in ix_score_latest_completed if datasheet.iloc[yt,0] in held_out_test_id]

np.random.choice(tech_ix_train_dev,(1,int(.20*len(tech_ix_train_dev))),replace=False)

## Dev set

In [ ]:
train_dev_ix = [t for t in tech_ix if t not in held_out_test_ix]

tech_ix_dev = np.random.choice(tech_ix_train_dev)

In [ ]:
dev_ix = np.array([460,  78, 234, 352, 347, 398, 448, 537])

In [ ]:
dev_id = [i for i in datasheet.iloc[dev_ix][0]]

In [ ]:
y_dev_ix = [yd for yd in ix_score_latest_completed if datasheet.iloc[yd,0] in dev_id]

## Train set

In [ ]:
train_ix = [i for i in train_dev_ix if i not in dev_ix]

In [ ]:
train_id = [i for i in datasheet.iloc[train_ix][0]]

In [ ]:
len(train_ix)

In [ ]:
train_ix_mem = [m for m in train_ix if m in memory_ix]

In [ ]:
len(train_ix_mem)

In [ ]:
y_train_ix = [yr for yr in ix_score_latest_completed if datasheet.iloc[yr,0] in train_id]

# Features

## Response length

### Response word count

In [ ]:
mem_n = pd.Series([word_count(r) for r in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
mean_mem_length = mean(mem_n)

In [ ]:
mean_mem_length

In [ ]:
tech_x_proxy_length = pd.Series([mean_mem_length for t in tech_x_ix],index=tech_x_ix)

In [ ]:
tech_x_proxy_length

mem_n = mem_n.append(tech_x_proxy_length)

#tech ix contains all respondent indices
len(mem_n) == len(tech_ix)

In [ ]:
tech_n = pd.Series([word_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Response sentence count

In [ ]:
mem_sentence = pd.Series([sentence_count(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

mean_mem_sentence = mean(mem_sentence)

tech_x_proxy_sentence = pd.Series([mean_mem_sentence for t in tech_x_ix],index=tech_x_ix)

mem_sentence = mem_sentence.append(tech_x_proxy_sentence)

len(mem_sentence) == len(tech_ix)

In [ ]:
tech_sentence = pd.Series([sentence_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
#And here you have it
#Merged response lengths
#sentences_count = pd.Series([mem_sentence[i]+tech_sentence[i] for i in tech_ix],index=tech_ix)

In [ ]:
#sentences_count.describe()

## Lexical Richness

## V

In [ ]:
mem_v = pd.Series([uniq_words(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_v = pd.Series([uniq_words(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## V1

In [ ]:
mem_v1 = pd.Series([single_appearances(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_v1 = pd.Series([single_appearances(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## MATTR (window size 10) 

In [ ]:
window_size = 10

In [ ]:
import statistics

In [ ]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]))]

In [ ]:
list_tokenize(datasheet.iloc[496,2])

In [ ]:
# mem_mattr = []
# for ix,mr in zip(memory_ix,datasheet.iloc[memory_ix,2]):
#     stripped_mr = list_tokenize(mr)
#     if len(stripped_mr) < 10: window_size = len(stripped_mr)
#     else: window_size = 10
        
#     try:
#         mem_mattr.append(mean([len(np.unique(mw))/window_size for mw in window(list_tokenize(mr),window_size)]))
#     except statistics.StatisticsError:
#         print('Stat error at ix',ix, mr)
# mem_mattr_a = pd.Series(mem_mattr,index=memory_ix)

In [ ]:
#ratio uniq words in each 10-word window
#avg for all ratios
mem_mattr = pd.Series([mean([len(np.unique(mw))/min(len(list_tokenize(mr)),10) for mw in window(list_tokenize(mr),n=min(len(list_tokenize(mr)),10))]) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
# mem_mattr_a.equals(mem_mattr)

In [ ]:
tech_mattr = pd.Series([mean([len(np.unique(tw))/min(len(list_tokenize(tr)),10) for tw in window(list_tokenize(tr),n=min(len(list_tokenize(tr)),10))]) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Rates

Noun

In [ ]:
mem_rate_nn = pd.Series([pos_rate(mr,nn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_rate_nn = pd.Series([pos_rate(tr,nn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Verb

In [ ]:
mem_rate_vb = pd.Series([pos_rate(mr,vb=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_rate_vb = pd.Series([pos_rate(tr,vb=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Pronoun

In [ ]:
mem_rate_pn = pd.Series([pos_rate(mr,pn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_rate_pn = pd.Series([pos_rate(tr,pn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix) 

## Initiating with Coordinating Conjunction
Counts, 0/1, rate-per-sentences?

In [ ]:
def coord_conjunctions_init(response):
    return sum([1 for sentence in ie_preprocess(response) if sentence[0][1] == 'CC'])

In [ ]:
mem_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Misspellings, words not found in dictionary

In [ ]:
def misspelt(response):
    return sum([1 for word in list_tokenize(response) if word.lower() not in word_list])

In [ ]:
mem_misspell = pd.Series([catch(lambda:misspelt(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_misspell = pd.Series([catch(lambda:misspelt(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Syntactic complexity

### Height of constituency tree, Stanford CoreNLP + nltk Tree

https://www.khalidalnajjar.com/setup-use-stanford-corenlp-server-python/


In [ ]:
'''
A sample code usage of the python package stanfordcorenlp to access a Stanford CoreNLP server.
Written as part of the blog post: https://www.khalidalnajjar.com/how-to-setup-and-use-stanford-corenlp-server-with-python/ 

cd stanford-corenlp-full-2018-10-05/ 
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -port 9000 -timeout 30000
'''

from stanfordcorenlp import StanfordCoreNLP
import logging
import json

class StanfordNLP:
    def __init__(self, host='http://localhost', port=9000):
        self.nlp = StanfordCoreNLP(host, port=port,
                                   timeout=30000)  # , quiet=False, logging_level=logging.DEBUG)
        self.props = {
            'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,dcoref,relation',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

    def word_tokenize(self, sentence):
        return self.nlp.word_tokenize(sentence)

    def pos(self, sentence):
        return self.nlp.pos_tag(sentence)

    def ner(self, sentence):
        return self.nlp.ner(sentence)

    def parse(self, sentence):
        return self.nlp.parse(sentence)

    def dependency_parse(self, sentence):
        return self.nlp.dependency_parse(sentence)

    def annotate(self, sentence):
        return json.loads(self.nlp.annotate(sentence, properties=self.props))

    @staticmethod
    def tokens_to_dict(_tokens):
        tokens = defaultdict(dict)
        for token in _tokens:
            tokens[int(token['index'])] = {
                'word': token['word'],
                'lemma': token['lemma'],
                'pos': token['pos'],
                'ner': token['ner']
            }
        return tokens

In [ ]:
from nltk.tree import Tree

In [ ]:
sNLP = StanfordNLP()

In [ ]:
mem_syntree = pd.Series([Tree.fromstring(str(sNLP.parse(mr))).height() for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_syntree = pd.Series([Tree.fromstring(str(sNLP.parse(tr))).height() for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Sentence complexity 

In [ ]:
scipy.stats.pearsonr(mem_syntree,tech_syntree[memory_ix])

In [ ]:
scipy.stats.skew(mem_syntree)

In [ ]:
scipy.stats.skew(tech_syntree)

In [ ]:
scipy.stats.kurtosis(mem_syntree)

In [ ]:
scipy.stats.kurtosis(tech_syntree)

In [ ]:
scipy.stats.spearmanr(mem_syntree,tech_syntree[memory_ix])

https://github.com/Lynten/stanford-corenlp/issues/59

## Reduced sentences

In [ ]:
# Reduced sentences:
#  Following the definition set out by [25], this feature represents those subordinated sentences without a conjunction but with nominal verb forms (which are either participles or gerund). To obtain the count for this feature, the frequencies of POS tags (VBG and VBN) are used.

# From <https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5237556/#CR25> 


In [ ]:
mem_reduced = pd.Series([sum([1 for (word,tag) in sNLP.pos(mr) if tag in ['VBG','VBN']]) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_reduced = pd.Series([sum([1 for (word,tag) in sNLP.pos(tr) if tag in ['VBG','VBN']]) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
scipy.stats.spearmanr(mem_reduced,tech_reduced[memory_ix])

## Sentiment analysis/Emotion recognition

In [ ]:
# http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [ ]:
vader

In [ ]:
for prompt in datasheet.iloc[memory_ix,2]:
    print(prompt)
    print(vader.polarity_scores(prompt))
    print('--------------------------------------\n')

In [ ]:
# One of my fondest childhood memories is how much I loved growing up around so much family.  My father's family all lived near us and we together frequently.  It was always so much happiness when everyone was together.
# {'neg': 0.0, 'neu': 0.782, 'pos': 0.218, 'compound': 0.8583}

In [ ]:
# I have a very clear memeory of last time I saw my father before he passed away in an accident. I was 14 years old.   I do not need to explain why this is important for me .
# {'neg': 0.084, 'neu': 0.788, 'pos': 0.128, 'compound': 0.1513}

In [ ]:
mem_sent_neg = pd.Series([vader.polarity_scores(prompt)['neg'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
mem_sent_neu = pd.Series([vader.polarity_scores(prompt)['neu'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
mem_sent_pos = pd.Series([vader.polarity_scores(prompt)['pos'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
mem_sent_cpd = pd.Series([vader.polarity_scores(prompt)['compound'] for prompt in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_sent_neg = pd.Series([vader.polarity_scores(prompt)['neg'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
tech_sent_neu = pd.Series([vader.polarity_scores(prompt)['neu'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
tech_sent_pos = pd.Series([vader.polarity_scores(prompt)['pos'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
tech_sent_cpd = pd.Series([vader.polarity_scores(prompt)['compound'] for prompt in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [ ]:
from textblob import TextBlob

In [ ]:
for blob in datasheet.iloc[memory_ix,2]:
    print(blob)
    print(TextBlob(blob).sentiment)

In [ ]:
mem_sent_pol = pd.Series([TextBlob(datasheet.iloc[mr,2]).sentiment[0] for mr in memory_ix],index=memory_ix)

In [ ]:
mem_sent_subj = pd.Series([TextBlob(datasheet.iloc[mr,2]).sentiment[1] for mr in memory_ix],index=memory_ix)

In [ ]:
tech_sent_pol = pd.Series([TextBlob(datasheet.iloc[tr,3]).sentiment[0] for tr in tech_ix],index=tech_ix)

In [ ]:
tech_sent_subj = pd.Series([TextBlob(datasheet.iloc[tr,3]).sentiment[1] for tr in tech_ix],index=tech_ix)

## Word frequency

## Lexical density - rate (nouns+vb+adj+adv)/n : (gets at semantic anal?)
file:///Users/renee/Downloads/2273-Article%20Text-4927-1-10-20100218.pdf
Lexical density is the term most often used for describing the proportion of
content words (nouns, verbs, adjectives, and often also adverbs) to the total
number of words. By investigating this, we receive a notion of information
packaging; a text with a high proportion of content words contains more
information than a text with a high proportion of function words (prepositions,
interjections, pronouns, conjunctions and count words). 

## Idea density 

https://github.com/andrecunha/idd3/blob/master/run.py

## SEMANTIC (high rate of function words? 

People + object reference words

In [ ]:
mem_rate_cloc = pd.Series([pos_rate(mr,cloc=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_rate_cloc = pd.Series([pos_rate(tr,cloc=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Stopwords

In [ ]:
help(stopwords)

In [ ]:
mem_rate_func = pd.Series([func_rate(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_rate_func = pd.Series([func_rate(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

# Feature Matrix

In [ ]:
mem_feature_list = [mem_n,mem_sentence,mem_v,mem_v1,mem_mattr,mem_rate_nn,mem_rate_vb,mem_rate_pn,mem_rate_cloc,mem_cc_init,mem_misspell,mem_syntree,mem_reduced,mem_sent_neg,mem_sent_neu,mem_sent_pos,mem_sent_cpd,mem_sent_pol,mem_sent_subj,mem_rate_func,mem_honore]

In [ ]:
mem_feature_df = pd.DataFrame(mem_feature_list).transpose()

In [ ]:
tech_feature_list = [tech_n,tech_sentence,tech_v,tech_v1,tech_mattr,tech_rate_nn,tech_rate_vb,tech_rate_pn,tech_rate_cloc,tech_cc_init,tech_misspell,tech_syntree,tech_reduced,tech_sent_neg,tech_sent_neu,tech_sent_pos,tech_sent_cpd,tech_sent_pol,tech_sent_subj,tech_rate_func,tech_honore]

In [ ]:
tech_feature_df = pd.DataFrame(tech_feature_list).transpose()

In [ ]:
X = mem_feature_df.join(tech_feature_df,how='outer',lsuffix='_mem_features',rsuffix='_tech_features')

Replace nan with col means

In [ ]:
X = X.fillna(X.mean())

## Feature Interaction

### Linear: Pearsons, Spearmans - helper function

In [ ]:
def pearsons_corr(y,X=X):
    pearsons_sig = []
    for col_name in X.columns:
        p_corr = scipy.stats.pearsonr(X[col_name],y)
        print('pearsons:',col_name,p_corr)
        if p_corr[1] < 0.05:
            pearsons_sig += [(col_name, p_corr)]
    return pearsons_sig

In [ ]:
def spearmans_corr(y,X=X):
    spearmans_sig = []
    for col_name in X.columns:
        s_corr = scipy.stats.spearmanr(X[col_name],y)
        print('spearmans:',col_name,s_corr)
        if s_corr[1] < 0.05:
            spearmans_sig += [(col_name,s_corr)]
    return spearmans_sig
        


### Interaction

https://chrisalbon.com/machine_learning/linear_regression/adding_interaction_terms/

### Clustering

### Information Gain

### Mutual Information

### Attribute interaction

### Bonferonni Adjustment

## Feature Correlation - Pearson's R [Linear Relationship] 

Skew

In [ ]:
X.columns = ['mem_n','mem_sentence','mem_v','mem_v1','mem_mattr','mem_rate_nn','mem_rate_vb','mem_rate_pn','mem_rate_cloc','mem_cc_init','mem_misspell','mem_syntree','mem_reduced','mem_sent_neg','mem_sent_neu','mem_sent_pos','mem_sent_cpd','mem_sent_pol','mem_sent_subj','mem_rate_func','mem_honore','tech_n','tech_sentence','tech_v','tech_v1','tech_mattr','tech_rate_nn','tech_rate_vb','tech_rate_pn','tech_rate_cloc','tech_cc_init','tech_misspell','tech_syntree','tech_reduced','tech_sent_neg','tech_sent_neu','tech_sent_pos','tech_sent_cpd','tech_sent_pol','tech_sent_subj','tech_rate_func','tech_honore']

In [ ]:
for col_name in X.columns:
    print(col_name,scipy.stats.skew(X[col_name]))

In [ ]:
for col_name in X.columns:
    print(col_name,scipy.stats.kurtosis(X[col_name]))

In [ ]:
X.corr()

In [ ]:
X.shape

## Target: Diagnosis

## y: clinical diagnosis

In [ ]:
lancet_target = [0 if record_id=='NL' else 1 for record_id in lancet_id_ix[17]]

In [ ]:
lancet_target = pd.Series(lancet_target,index=lancet_id_ix[0])

In [ ]:
#order target by X.index
#ix_id_series indexed by X.index
#lancet_target.loc indexed by ix_id_series record_id
y_diagnosis = [lancet_target.loc[ix_id_series[ix]] for ix in X.index]

In [ ]:
#SCI considered cognitive typical, 0
lancet_target_sci_ct = [0 if record_id=='NL' or record_id =='SCI' else 1 for record_id in lancet_id_ix[17]]

In [ ]:
lancet_target_sci_ct = pd.Series(lancet_target_sci_ct,index=lancet_id_ix[0])

In [ ]:
y_sci_ct = pd.Series([lancet_target_sci_ct.loc[ix_id_series[ix]] for ix in X.index],index=X.index)

In [ ]:
p_sig_diagnosis = pearsons_corr(y_sci_ct)

In [ ]:
p_sig_diagnosis

In [ ]:
s_sig_diagnosis = spearmans_corr(y_sci_ct)

In [ ]:
s_sig_diagnosis

### y: MMSE

In [ ]:
y_mmse = pd.Series([y for y in datasheet.iloc[ix_score_latest_completed,74]])

In [ ]:
y_mmse = y_mmse.fillna(math.floor(y_mmse.mean()))

In [ ]:
p_sig_mmse = pearsons_corr(y_mmse)

In [ ]:
p_sig_mmse

In [ ]:
s_sig_mmse = spearmans_corr(y_mmse)

In [ ]:
s_sig_mmse

## Targets: Language + Verbal Knowledge

### y: Picture vocab

In [ ]:
y_pic_vocab = datasheet.iloc[ix_score_latest_completed,9]

In [ ]:
y_pic_vocab.describe()

In [ ]:
s_sig_picvocab = spearmans_corr(y_pic_vocab)

In [ ]:
s_sig_picvocab

In [ ]:
p_sig_picvocab = pearsons_corr(y_pic_vocab)

In [ ]:
p_sig_picvocab

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

### y: Oral reading

In [ ]:
y_oral_reading = datasheet.iloc[ix_score_latest_completed,12]

In [ ]:
p_sig_orreading = pearsons_corr(y_oral_reading)

In [ ]:
p_sig_orreading

In [ ]:
s_sig_orreading = spearmans_corr(y_oral_reading)

In [ ]:
s_sig_orreading

In [ ]:
y_oral_reading.describe()

In [ ]:
mem_rate_cloc

In [ ]:
y_oral_reading.std()

In [ ]:
np.mean(y_oral_reading)

In [ ]:
min(y_oral_reading)